In [1]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

class RBF(nn.Module):
    def __init__(self):
        super(RBF, self).__init__()
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        torch.cuda.manual_seed(0)
        self.cause_clt = self.init_clt()
        self.cause_std = self.init_std()       
        
    def init_clt(self):
        return nn.Parameter(torch.rand(1, device=device))

    def init_std(self):
        return nn.Parameter(torch.rand(1, device=device))
    
    def rbf(self, x, cluster, std):
        return -(x - cluster) * (x - cluster) / 2 * (std * std)
    
    def forward(self, t):
        for i in range(t):
            if i == 0:
                a = self.rbf(i, self.cause_clt, self.cause_std)
            else:
                a = torch.cat([a, self.rbf(i, self.cause_clt, self.cause_std)], dim=0)
                
        return a

In [2]:

def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params
        
def train_rbf(model, Y, lr, epochs, lookback=5,device = device):

    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loss_list = []

    best_it = None
    best_model = None
    best_loss = np.inf

    for epoch in range(epochs):        
        # pred loss
        pred = model(len(Y))
        loss = sum([loss_fn(pred[i], Y[i]) for i in range(len(Y))])
        
        if epoch % 100 == 0:
            print("epoch {} loss {} :".format(epoch, loss / len(Y)))
            print("------------------------------------------------------")
            print()
                      
        loss.backward()
        optimizer.step()
        model.zero_grad()
        mean_loss = loss / len(Y)
        train_loss_list.append(mean_loss)
        
        
        if mean_loss < best_loss:
            best_loss = mean_loss
            best_it = epoch
            best_model = deepcopy(model)
            
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break

    restore_parameters(model, best_model)

    return train_loss_list , model

In [5]:
import pandas as pd
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')
a = df['a'].values
a = torch.tensor(a, device=device).float()

In [6]:
model = RBF()
print(model.cause_clt)
print(model.cause_std)
loss, best_model = train_rbf(model, a, 0.001, 10000, device)
print(best_model.cause_clt)
print(best_model.cause_std)

Parameter containing:
tensor([0.3990], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.9722], device='cuda:0', requires_grad=True)
epoch 0 loss 44475576320.0 :
------------------------------------------------------

epoch 100 loss 29498050560.0 :
------------------------------------------------------

epoch 200 loss 19997001728.0 :
------------------------------------------------------

epoch 300 loss 13874695168.0 :
------------------------------------------------------

epoch 400 loss 9840637952.0 :
------------------------------------------------------

epoch 500 loss 7123165184.0 :
------------------------------------------------------

epoch 600 loss 5253264896.0 :
------------------------------------------------------

epoch 700 loss 3940527872.0 :
------------------------------------------------------

epoch 800 loss 3001554688.0 :
------------------------------------------------------

epoch 900 loss 2318248704.0 :
------------------------------------------

KeyboardInterrupt: 

In [ ]:
44475576320
29498050560
  132210328
    2285494